In [20]:
import py3Dmol
from rdkit import Chem
import py3Dmol
from rdkit import Chem
import ast

def draw(sdf_file, def_order=[], id=0):
    suppl = Chem.SDMolSupplier(sdf_file, removeHs=False)
    mol = suppl[id] if suppl else None

    if mol:
        mol_block = Chem.MolToMolBlock(mol)
        viewer = py3Dmol.view(width=400, height=400)
        viewer.addModel(mol_block, "mol")  # Add molecule model
        viewer.setStyle({"stick": {}})  # Stick representation
        
        # Add title label for atom labels.
        viewer.addLabel(".", {"position": {"x": 0, "y": 0, "z": 0},
                                          "backgroundColor": "black",
                                          "fontColor": "white"})

        # Try to extract the SMILES order property if it exists.
        try:
            smiles_order1 = list(map(int, ast.literal_eval(mol.GetProp('_smilesAtomOutputOrder'))))
        except Exception:
            smiles_order1 = def_order
            print("No SMILES order property found in SDF file.")
        smiles_order = [0] * len(smiles_order1)
        for smiles_id, sdf_id in enumerate(smiles_order1):
            smiles_order[sdf_id] = smiles_id
        for atom in mol.GetAtoms():
            pos = mol.GetConformer().GetAtomPosition(atom.GetIdx())
            # Build the label: include the atom symbol and its SMILES order if available.
            atom_label = atom.GetSymbol()
            if smiles_order is not None and atom.GetIdx() < len(smiles_order):
                atom_label += f"{smiles_order[atom.GetIdx()]}"
            viewer.addLabel(atom_label,
                            {"position": {"x": pos.x, "y": pos.y, "z": pos.z},
                             "backgroundColor": "gray",
                             "fontColor": "white"})

        viewer.zoomTo()  # Auto zoom
        viewer.show()

    else:
        print("No valid molecules found in SDF file.")
        
# def draw(sdf_file, id=0):
#     suppl = Chem.SDMolSupplier(sdf_file, removeHs = False)
#     mol = suppl[id] if suppl else None

#     if mol:
#         mol_block = Chem.MolToMolBlock(mol)
#         viewer = py3Dmol.view(width=400, height=400)
#         viewer.addModel(mol_block, "mol")  # Add molecule model
#         viewer.setStyle({"stick": {}})  # Stick representation
        
#         # Add atom labels
#         viewer.addLabel("Atom Labels", {"position": {"x": 0, "y": 0, "z": 0}, "backgroundColor": "black", "fontColor": "white"})  # Title label
#         for atom in mol.GetAtoms():
#             pos = mol.GetConformer().GetAtomPosition(atom.GetIdx())
#             viewer.addLabel(atom.GetSymbol(), {"position": {"x": pos.x, "y": pos.y, "z": pos.z}, "backgroundColor": "gray", "fontColor": "white"})

#         viewer.zoomTo()  # Auto zoom
#         viewer.show()

#     else:
#         print("No valid molecules found in SDF file.")


In [22]:
sdf_file = "/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/special_mol1.sdf"
draw(sdf_file, [15, 14, 16, 17, 12, 13, 11, 10, 9, 18, 19, 3, 8, 4, 5, 6, 7, 0, 1, 2, 20])
sdf_file = "/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/special-rec1.sdf"
draw(sdf_file, [i for i in range(21)])

No SMILES order property found in SDF file.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

No SMILES order property found in SDF file.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
sdf_file = "/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/special_mol1.sdf"
draw(sdf_file, [5, 4, 3, 2, 6, 7, 8, 9, 10, 11, 1, 0, 12])
sdf_file = "/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/special-rec.sdf"
draw(sdf_file, [i for i in range(13)])

No SMILES order property found in SDF file.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

No SMILES order property found in SDF file.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
draw("/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/error.sdf", 0)
draw("/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/error.sdf", 1)
draw("/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/error.sdf", 2)

In [34]:
sdf_file = "/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/special_mol.sdf"
draw(sdf_file)

In [35]:
sdf_file = "/auto/home/filya/3DMolGen/molgen3D/data_processing/molecules/special-rec.sdf"
draw(sdf_file)